<center>
    <h1>Interactive Demos of C++ Libraries in the Browser</h1>
    <h3>Powered by Xeus-Cpp-Lite and WebAssembly</h3>
</center>

This notebook demonstrates several powerful C++ libraries running entirely in the browser using **xeus-cpp-lite**. The examples span symbolic computation, numerical analysis, interactive visualization, and more — all compiled to WebAssembly and ready to run in a JupyterLite environment.

**Featured libraries:**

1. **Scientific Computation (Symbolic & Numeric)**: `boost`, `symengine`, `openblas`
2. **Array based Computation through Xtensor-stack**: `xtl`, `xtensor`, `xsimd`, `xtensor-blas`
3. **Interactive Widgets**: `xwidgets`, `xcanvas`
4. **Utilities & Miscellaneous**: `nlohmann_json`, and others

## Scientific Computation (Symbolic & Numeric)

Let’s begin with powerful C++ libraries for symbolic math, high-precision arithmetic, and efficient numerical computation using BLAS and LAPACK routines.

1. [boost-cpp](https://github.com/boostorg/boost): Collection of peer-reviewed portable C++ libraries for tasks like linear algebra, multithreading, networking, parsing, and more.
2. [symengine](https://github.com/symengine/symengine): Fast C++ symbolic mathematics library supporting algebraic manipulation, calculus, equation solving, and more, with optional Python bindings.

In [1]:
#include <boost/multiprecision/cpp_int.hpp>
using namespace boost::multiprecision;
using namespace std;

int128_t boost_product(long long A, long long B)
{
    int128_t ans = (int128_t)A * B;
    return ans;
}

long long first = 98745636214564698;
long long second = 7459874565236544789;
cout << "Product of " << first << " * " << second << " = \n"
     << boost_product(first, second);

Product of 98745636214564698 * 7459874565236544789 = 
736630060025131838840151335215258722

In [2]:
#include <symengine/expression.h>

using namespace SymEngine;

Expression x("x");
auto ex = pow(x + sqrt(Expression(2)), 6);

In [3]:
#include <xcpp/xdisplay.hpp>
xcpp::display(ex);

(x + sqrt(2))**6

In [4]:
auto expanded_ex = expand(ex);
xcpp::display(expanded_ex);

8 + 24*sqrt(2)*x + 40*sqrt(2)*x**3 + 6*sqrt(2)*x**5 + 60*x**2 + 30*x**4 + x**6

## Array based Computation through Xtensor-stack

Explore NumPy-style array computing in C++ with Xtensor, Xtensor-BLAS, and Xsimd — enabling high-performance numerical and SIMD-accelerated workflows.

1. [openblas](https://github.com/xianyi/OpenBLAS): Optimized BLAS and LAPACK implementation providing efficient linear algebra routines such as matrix multiplication, factorization, and solvers.
2. [xtensor](https://github.com/xtensor-stack/xtensor): C++ multidimensional array library with NumPy-like syntax, enabling numerical computing, broadcasting, and integration with C++ scientific libraries.
3. [xtensor-blas](https://github.com/xtensor-stack/xtensor-blas): BLAS and LAPACK bindings for xtensor, adding high-performance linear algebra operations like dot products, decompositions, and solvers.
4. [xsimd](https://github.com/xtensor-stack/xsimd): C++ library providing a high-level SIMD API that abstracts platform-specific vectorization instructions, simplifying portable and efficient numerical code.


In [5]:
#include <iostream>
#include <iomanip>

typedef int INTEGER;
typedef double DOUBLE;

extern "C" {
    int dgetrf_(const INTEGER* m, const INTEGER* n, DOUBLE* a,
                const INTEGER* lda, INTEGER* ipiv, INTEGER* info);
}

const INTEGER m = 3, n = 3, lda = 3;
DOUBLE A[9] = {
    1.0, 2.0, 3.0,
    4.0, 5.0, 6.0,
    7.0, 8.0, 10.0
};  // Column-major layout

INTEGER ipiv[3];
INTEGER info;

dgetrf_(&m, &n, A, &lda, ipiv, &info);

// Output
std::cout << std::fixed << std::setprecision(6);
std::cout << "LU Decomposition (OpenBLAS dgetrf_)\n";
std::cout << "info: " << info << "\n\n";

std::cout << "Factorized Matrix (A after dgetrf_):\n";
for (int i = 0; i < m; ++i) {
    for (int j = 0; j < n; ++j)
        std::cout << std::setw(10) << A[i + j * lda];
    std::cout << "\n";
}

std::cout << "\nPivot Indices:\n";
for (int i = 0; i < std::min(m, n); ++i)
    std::cout << ipiv[i] << " ";
std::cout << "\n";


LU Decomposition (OpenBLAS dgetrf_)
info: 0

Factorized Matrix (A after dgetrf_):
  3.000000  6.000000 10.000000
  0.333333  2.000000  3.666667
  0.666667  0.500000 -0.500000

Pivot Indices:
3 3 3 


In [6]:
?xt::xtensor

https://xtensor.readthedocs.io/en/latest/api/xtensor.html#_CPPv2N2xt7xtensorE

In [7]:
#include "xtensor/containers/xarray.hpp"
#include "xtensor/io/xio.hpp"
#include "xtensor/views/xview.hpp"

xt::xarray<double> arr1 = {{1.0, 2.0, 3.0}, {2.0, 5.0, 7.0}, {2.0, 5.0, 7.0}};
xt::xarray<double> arr2{5.0, 6.0, 7.0};

xcpp::display(xt::view(arr1, 1) + arr2);

7.
11.
14.


In [8]:
xt::xarray<int> arr = {1, 2, 3, 4, 5, 6, 7, 8, 9};
arr.reshape({3, 3});

xcpp::display(arr);

1,2,3
4,5,6
7,8,9


In [9]:
#include <iostream>
#include "xtensor-blas/xlinalg.hpp"

xt::xtensor<double, 2> M = {{1.5, 0.5}, {0.7, 1.0}};

In [10]:
std::cout << "Matrix rank: " << xt::linalg::matrix_rank(M) << std::endl;
std::cout << "Matrix inverse: " << std::endl;
xcpp::display(xt::linalg::inv(M));
std::cout << "Eigen values: " << std::endl;
xcpp::display(xt::linalg::eigvals(M));

Matrix rank: 2
Matrix inverse: 


0.869565,-0.434783
-0.608696,1.304348


Eigen values: 


1.892262+0.i
0.607738+0.i


In [11]:
#include <xsimd/xsimd.hpp>

namespace xs = xsimd;

// Define two SIMD float vectors using the wasm backend
xs::batch<float, xs::wasm> X = {1.2f, 3.4f, 5.6f, 7.8f};
xs::batch<float, xs::wasm> Y = {2.1f, 4.3f, 6.5f, 8.7f};

// Perform SIMD addition
xs::batch<float, xs::wasm> Z = X + Y;

// Print the result
std::cout << Z << std::endl;

(3.300000, 7.700000, 12.100000, 16.500000)


## Interactive Widgets

Build rich, interactive UI components in C++ using xwidgets and xcanvas, seamlessly integrated with the Jupyter frontend.

1. [xwidgets](https://github.com/jupyter-xeus/xwidgets): C++ implementation of the Jupyter widget protocol, providing core interactive widgets and serving as the foundation for libraries like xleaflet and xcanvas.
2. [xcanvas](https://github.com/jupyter-xeus/xcanvas): C++ library bringing the HTML5 canvas API to Jupyter via the widget protocol, enabling interactive 2D graphics and visualizations in notebooks.

In [12]:
#include <xwidgets/xslider.hpp>

xw::slider<double> slider;

In [13]:
slider.value = 20;
slider.max = 40;
slider.style().handle_color = "blue";
slider.orientation = "vertical";
slider.description = "A slider";

In [14]:
// Need to re-run this to see the slider being displayed
xcpp::display(slider);

A Jupyter widget with unique id: 2f0bf4f323ea4ccdb9337a9115e4320d

In [15]:
#include <sstream>
#include "xcanvas/xcanvas.hpp"

auto canvas = xc::canvas().initialize()
    .width(300)
    .height(300)
    .finalize();

In [16]:
canvas.shadow_offset_x = 0;
canvas.shadow_offset_y = 0;
canvas.shadow_blur = 0;

for (std::size_t i = 0; i < 200; i++)
{
    std::stringstream color;
    color << "rgb(200, " << 200 - i << ", " << i << ")";
    canvas.fill_style = color.str();
    canvas.fill_rect(0, 0, 200, 200 - i);
}

In [17]:
// Need to re-run this to see the canvas being displayed
xcpp::display(canvas);

A Jupyter widget with unique id: 39f80e8c2a204499bc095f1defdce65f

In [18]:
#include <emscripten.h>

int width2 = 400;
int height2 = 400;

auto canvas2 = xc::canvas().initialize()
    .width(width2)
    .height(height2)
    .finalize();

using canvas2_type = std::decay_t<decltype(canvas2)>;

xcpp::display(canvas2);

struct Context2
{
    canvas2_type *canvas = nullptr;
    int iter = 0;
    int stop = 100000;
    int width = 400;
    int height = 400;

    // the on_mouse_move callbacks would set these
    int mouse_pos_x = 200;
    int mouse_pos_y = 200;
    int mouse_is_down = 0;
};

void callback2(void* arg) {
    Context2 &ctx = *reinterpret_cast<Context2*>(arg);

    if (ctx.iter >= ctx.stop)
    {
        emscripten_cancel_main_loop();
    }

    // clear the canvas
    ctx.canvas->clear_rect(0, 0, ctx.width, ctx.height);

    std::stringstream color_ss;
    color_ss << "rgba(" << (ctx.iter % 255) << ", "
             << (ctx.iter * 2 % 255) << ", "
             << (ctx.iter * 3 % 255) << ", 0.5)";

    // draw a circle at the mouse position
    ctx.canvas->begin_path();
    ctx.canvas->arc(ctx.mouse_pos_x, ctx.mouse_pos_y,
                    5 + (ctx.iter / 3) % 20, 0, 2 * M_PI);
    ctx.canvas->fill_style = color_ss.str();
    ctx.canvas->fill();

    ctx.iter++;
}

// type of the callback
using xy_callback_type2 = std::function<void(int, int)>;

Context2 ctx2{};
auto ctx2_ptr = &ctx2;

xy_callback_type2 cb2_on_mouse_move([](int x, int y) {
    ctx2_ptr->mouse_pos_x = x;
    ctx2_ptr->mouse_pos_y = y;
});

xy_callback_type2 cb2_on_mouse_down([](int x, int y) {
    ctx2_ptr->mouse_is_down = 1;
    ctx2_ptr->mouse_pos_x = x;
    ctx2_ptr->mouse_pos_y = y;
});

xy_callback_type2 cb2_on_mouse_up([](int x, int y) {
    ctx2_ptr->mouse_is_down = 0;
    ctx2_ptr->mouse_pos_x = x;
    ctx2_ptr->mouse_pos_y = y;
});

xy_callback_type2 cb2_on_mouse_leave([](int x, int y) {
    ctx2_ptr->mouse_is_down = 0;
    ctx2_ptr->mouse_pos_x = x;
    ctx2_ptr->mouse_pos_y = y;
});

canvas2.on_mouse_move(cb2_on_mouse_move);
canvas2.on_mouse_down(cb2_on_mouse_down);
canvas2.on_mouse_up(cb2_on_mouse_up);
canvas2.on_mouse_leave(cb2_on_mouse_leave);

ctx2.canvas = &canvas2;
ctx2.width = width2;
ctx2.height = height2;

void* void_ctx2 = reinterpret_cast<void*>(&ctx2);
emscripten_set_main_loop_arg(callback2, void_ctx2, 0, false);

A Jupyter widget with unique id: a33b74e244e04db2931d1992de9e2817

## Magic Commands

Magics are special commands assisting code execution for the user. Specific to the kernel and are not a part of the C++ programming language.

There are defined with the symbol % for a line magic and %% for a cell magic.

1. `%%file`: Open, read, write to a file.  
2. `%mamba`: Fetch packages from emscripten-forge at runtime.

In [19]:
%%file tmp.txt
Demo of magic command

Writing tmp.txt


In [20]:
%%file -a tmp.txt
append at the end

Appending to tmp.txt


In [21]:
#include <fstream>

std::ifstream infile("tmp.txt");
std::string line;
while (std::getline(infile, line)) {
    std::cout << line << std::endl;
}
infile.close();

Demo of magic command
append at the end


In [22]:
%mamba install doctest

Specs: xeus-cpp, symengine, xtensor-blas, xsimd, xwidgets, xcanvas, doctest
Channels: https://repo.prefix.dev/emscripten-forge-dev, conda-forge

Solving environment...
Solving took 7.976399999976159 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ doctest                       2.4.12                        h2072262_0                    https://repo.prefix.dev/emscripten-forge-dev/
- pip                           25.2                          pyh145f28c_0                  conda-forge                   


In [23]:
#define DOCTEST_CONFIG_IMPLEMENT
#include <doctest/doctest.h>

TEST_CASE("Simple check") {
    CHECK(1 == 2);
}

int main() {
    doctest::Context context;
    context.setOption("success", true); // Show successful tests
    return context.run();
}

In [24]:
main();

[doctest] doctest version is "2.4.12"
[doctest] run with "--help" for options
input_line_21:4:
TEST CASE:  Simple check

input_line_21:5: ERROR: CHECK( 1 == 2 ) is NOT correct!
  values: CHECK( 1 == 2 )

[doctest] test cases: 1 | 0 passed | 1 failed | 0 skipped
[doctest] assertions: 1 | 0 passed | 1 failed |
[doctest] Status: FAILURE!


## Miscallaneous

Feel free to explore other useful C++ libraries in the browser, like nlohmann_json, pugixml, xtl etc

In [25]:
#include "nlohmann/json.hpp"

nlohmann::json jsonObject;

jsonObject["name"] = "John Doe";
jsonObject["age"] = 30;
jsonObject["is_student"] = false;
jsonObject["skills"] = {"C++", "Python", "JavaScript"};

std::cout << jsonObject.dump(4) << std::endl;

{
    "age": 30,
    "is_student": false,
    "name": "John Doe",
    "skills": [
        "C++",
        "Python",
        "JavaScript"
    ]
}
